In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [9]:
# Hyper parameters
sequence_length = 28
input_size = 28
hidden_size = 150
num_layers = 2
output_classes = 10
num_epochs = 2
learning_rate = 0.001
batch_size = 20

In [10]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [11]:
# Recurrent Neural Network
class RNN(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, output_classes):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.output_classes = output_classes
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.linear(out[:, -1, :])
        return out

In [12]:
model = RNN(input_size, num_layers, hidden_size, output_classes)

In [13]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [14]:
# Train the model
model.train()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/3000], Loss: 1.2475
Epoch [1/2], Step [200/3000], Loss: 0.8809
Epoch [1/2], Step [300/3000], Loss: 1.1260
Epoch [1/2], Step [400/3000], Loss: 0.4475
Epoch [1/2], Step [500/3000], Loss: 0.8161
Epoch [1/2], Step [600/3000], Loss: 0.1506
Epoch [1/2], Step [700/3000], Loss: 0.2945
Epoch [1/2], Step [800/3000], Loss: 0.2164
Epoch [1/2], Step [900/3000], Loss: 0.2239
Epoch [1/2], Step [1000/3000], Loss: 0.4063
Epoch [1/2], Step [1100/3000], Loss: 0.3519
Epoch [1/2], Step [1200/3000], Loss: 0.5625
Epoch [1/2], Step [1300/3000], Loss: 0.0898
Epoch [1/2], Step [1400/3000], Loss: 0.5908
Epoch [1/2], Step [1500/3000], Loss: 0.1882
Epoch [1/2], Step [1600/3000], Loss: 0.4072
Epoch [1/2], Step [1700/3000], Loss: 0.2491
Epoch [1/2], Step [1800/3000], Loss: 0.2366
Epoch [1/2], Step [1900/3000], Loss: 0.4243
Epoch [1/2], Step [2000/3000], Loss: 0.3039
Epoch [1/2], Step [2100/3000], Loss: 0.0188
Epoch [1/2], Step [2200/3000], Loss: 0.2730
Epoch [1/2], Step [2300/3000], Loss: 0.39

In [15]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size)
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 91.85 %
